# Modelling v5

modelling use the latest exploration

# Goal

Get the model from the prepared data. With undersampling + sample weight. 

# Plan

Checklist what will be done on the notebook :

    [*] Get Data
    [*] Modelling



In [1]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import os
import joblib
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic
import time

from scipy.stats import uniform,randint
import warnings
warnings.filterwarnings("ignore")

In [2]:
if 'iter' in os.getcwd().split('/')[-1]:
    ######## run manually #############
    with open("../../config.yaml", "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
        
    if os.name == 'posix' :
        PATH_MODEL = config['path']['artifact']+os.getcwd().split('/')[-1]+'/'
    else:
        PATH_MODEL = config['path']['artifact']+os.getcwd().split('\\')[-1]+'/'

else:
    ######### run with makefile #############
    with open("config-make.yaml", "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)

    if os.name == 'posix' :
        PATH_MODEL = config['path']['artifact']+'iteration_2/'
    else:
        PATH_MODEL = config['path']['artifact']+'iteration_2/'

PATH_RAW = config['path']['raw']
PATH_INTERIM = config['path']['interim']
PATH_PROCESSED = config['path']['processed']
PATH_UDF = config['path']['udf']

In [3]:
from sklearn.linear_model import LogisticRegression, ElasticNet, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessRegressor

## Get Data

In [4]:
raw_data = pd.read_csv(PATH_PROCESSED+'prep_iter2.csv')
col_cat_load = joblib.load(PATH_MODEL+'cat_names.pkl')
col_num_load = joblib.load(PATH_MODEL+'num_names.pkl')

In [5]:
all_tr = raw_data[raw_data.is_train == 1]
down_size = int(len(all_tr[all_tr.target == 1]) * 2)
all_tr0 = all_tr[all_tr.target == 0].sample(down_size,random_state = 125)

all_tr0['sweight'] = 0.333/0.116
all_tr1 = all_tr[all_tr.target == 1]
all_tr1['sweight'] = 1

all_tr_down = pd.concat([all_tr0,all_tr1])
all_tr_down.target.value_counts(normalize = 1)

samp_w = all_tr_down['sweight']

In [6]:
X_tr = all_tr_down[col_num_load+col_cat_load]
Y_tr = all_tr_down['target'].values

X_val = raw_data[raw_data.is_train == 0][col_num_load+col_cat_load]
Y_val = raw_data[raw_data.is_train == 0]['target'].values

In [7]:
X_tr.shape

(11316, 26)

In [8]:
X_val.shape

(5765, 26)

In [9]:
Y_val.shape

(5765,)

In [10]:
Y_tr.mean()

0.3333333333333333

In [11]:
samp_w

5079     2.87069
17291    2.87069
9206     2.87069
4581     2.87069
19666    2.87069
          ...   
38422    1.00000
38424    1.00000
38425    1.00000
38426    1.00000
38427    1.00000
Name: sweight, Length: 11316, dtype: float64

## Modelling

In [12]:
proportion_cls = pd.Series(Y_tr).value_counts(normalize = True)
scale_pos_w = np.ceil(proportion_cls[0]/proportion_cls[1])
cw = {0:1, 1:scale_pos_w}
cw
# scale_pos_weight for xgboost

{0: 1, 1: 2.0}

In [13]:
proportion_cls

0    0.666667
1    0.333333
dtype: float64

In [14]:
base_logreg = LogisticRegression(C=1.9527291, class_weight=cw,solver='sag')
all_model = {'sgd5' : SGDClassifier(class_weight=cw,max_iter = 5000,early_stopping=True,warm_start=True,eta0=0.01)
            ,'logreg5' : LogisticRegression(class_weight=cw)
            ,'rf5': RandomForestClassifier(random_state=234) ## not use class weight
            ,'nb5':GaussianNB(priors=[0.6667,0.3333])
            ,'baglog5': BaggingClassifier(base_estimator=base_logreg, random_state=234)
            ,'adab_tree5':AdaBoostClassifier(random_state=234)
            }

In [15]:
param_options = {'logreg5' : dict(C=uniform(loc=0, scale=2)
                                 ,penalty=['l2', 'l1']
                                 ,solver=['liblinear', 'sag'])
                 ,'rf5' : dict(n_estimators=randint(300, 500)
                             ,max_depth=randint(20, 50)
                             ,min_samples_split=randint(5, 20)
                             ,min_samples_leaf=randint(5, 20)
                             ,min_weight_fraction_leaf=uniform(loc=0, scale=0.4)
                             ,min_impurity_decrease=uniform(loc=0, scale=0.3))
                 ,'nb5' : dict(var_smoothing = uniform(loc=1e-8, scale=1e-2))
                 ,'sgd5' : dict(loss=['hinge', 'squared_hinge', 'perceptron']
                                ,penalty=['l2', 'l1']
                                ,alpha=uniform(loc=0.1, scale=0.95)
                                ,l1_ratio=uniform(loc=0.1, scale=0.9)
                                ,learning_rate=['constant','optimal','invscaling','adaptive'])
                 ,'baglog5' : dict(n_estimators=randint(4, 45)
                                   ,max_samples=uniform(loc=0.3,scale=0.7)
                                   ,max_features=uniform(loc=0.3,scale=0.7)
                                   ,warm_start=[False,True])
                 ,'adab_tree5' : dict(n_estimators=randint(4, 100)
                                   ,learning_rate=uniform(loc=0.3,scale=0.7)
                                  )
                }

 # min_weight_fraction_leaf must in [0, 0.5]

In [1]:
## scoring
## https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
best_clf = {}
best_scr = {}
sla_model = {}
for mdl in all_model.keys():
    L1 = time.time()
    clf = all_model[mdl]
    parm = param_options[mdl]
    RS = RandomizedSearchCV(clf, parm, n_iter = 300, cv = 5, scoring = 'roc_auc',n_jobs = 10)
        
    RS.fit(X_tr,Y_tr, sample_weight=samp_w)
    
    best_clf[mdl] = RS.best_estimator_
    best_scr[mdl] = RS.best_score_
    diff_time = round((time.time()-L1)/60, 2)
    
    joblib.dump(best_clf[mdl],PATH_MODEL+mdl+'.pkl')
    
    print(mdl, 'is done in ',diff_time, ' minutes')
    sla_model[mdl] = diff_time

In [17]:
sla_model

{'sgd5': 0.3,
 'logreg5': 1.16,
 'rf5': 4.77,
 'nb5': 0.11,
 'baglog5': 30.69,
 'adab_tree5': 1.9}

In [18]:
best_scr

{'sgd5': 0.834447230809346,
 'logreg5': 0.8300272223819182,
 'rf5': 0.8085655407819272,
 'nb5': 0.759538962755866,
 'baglog5': 0.8372180899763702,
 'adab_tree5': 0.8434293655449796}

In [19]:
best_clf

{'sgd5': SGDClassifier(alpha=0.8573936094816896, class_weight={0: 1, 1: 2.0},
               early_stopping=True, eta0=0.01, l1_ratio=0.5412478934296541,
               learning_rate='adaptive', max_iter=5000, warm_start=True),
 'logreg5': LogisticRegression(C=0.05326218898689827, class_weight={0: 1, 1: 2.0},
                    penalty='l1', solver='liblinear'),
 'rf5': RandomForestClassifier(max_depth=30, min_impurity_decrease=0.003454023513384952,
                        min_samples_leaf=18, min_samples_split=10,
                        min_weight_fraction_leaf=0.010148696237279743,
                        n_estimators=360, random_state=234),
 'nb5': GaussianNB(priors=[0.6667, 0.3333], var_smoothing=0.007980873590410475),
 'baglog5': BaggingClassifier(base_estimator=LogisticRegression(C=1.9527291,
                                                     class_weight={0: 1, 1: 2.0},
                                                     solver='sag'),
                   max_features=0.9971

## Evaluation

The SVM is best so far, but there is no predict proba.

In [20]:
for ml in best_clf.keys():
    Y_prob_pred = best_clf[ml].predict(X_val)
    print(ml)
    print(classification_report(Y_val,Y_prob_pred))

sgd5
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      5079
           1       0.00      0.00      0.00       686

    accuracy                           0.88      5765
   macro avg       0.44      0.50      0.47      5765
weighted avg       0.78      0.88      0.83      5765

logreg5
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      5079
           1       0.48      0.51      0.50       686

    accuracy                           0.88      5765
   macro avg       0.71      0.72      0.71      5765
weighted avg       0.88      0.88      0.88      5765

rf5
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      5079
           1       0.00      0.00      0.00       686

    accuracy                           0.88      5765
   macro avg       0.44      0.50      0.47      5765
weighted avg       0.78      0.88      0.83      5765

nb

In [21]:
1-Y_val.mean()

0.8810060711188205